2021.10.09 THIS DOES NOT WORK OUT OF THE BOX
* this is still sandbox-y and file paths have been shifted around from restructuring
* general idea is
  * open main dictionary .json
  * parse the .json and build up dictionary entries in html
  * replace the @BODY of some content_template.html
  * write file to content.html

In [ ]:
import json
import os
import re

## Open main dict_concised.json file


In [ ]:
f = open('dict-concised.json', 'r', encoding='utf-8')
data = json.load(f)
f.close()
print(len(data))

Method 1
* no indenting, old headers
* but has custom ordered list (except doesn't check if had leading number so will add number to non numbered lists)

In [ ]:
from yattag import Doc
from yattag import indent

doc, tag, text, line = Doc().ttl()

for j in range(len(data)):
    print(j)
    entry = data[j]
    # 'title' - searched character(s)
    if '{[' not in entry['title']:
        single_char = False
        
        if len(entry['title']) == 1:
            single_char = True
            if 'radical' in entry:
                radical = entry['radical']
            if 'non_radical_stroke_count' in entry:
                non_radical_stroke_count = entry['non_radical_stroke_count']
            if 'stroke_count' in entry:
                stroke_count = entry['stroke_count']
        
        # create entry
        with tag('idx:entry', name="default", scriptable="yes", spell="yes"):
            # create title
            with tag('h5'):
                # custom styling for title
                with tag('dt'):
                    with tag('idx:orth'):
                        text(entry['title'])
                    # add extra info for single characters
                    if single_char:
                        text('    (部首：{}, 部首外筆畫數：{}, 總筆畫數：{})'.format(radical, non_radical_stroke_count, stroke_count))
            # create definition for each heteronym (pronunciation -- typically more for single characters)
            for h in entry['heteronyms']:
                # custom styling for heteronym
                with tag('dh'):
                    # add pronunciation if exists (assume bpmf and pinyin always come together)
                    if 'bopomofo' in h and 'pinyin' in h:
                        text(h['bopomofo'])
                        doc.asis('<br>') 
                        text(h['pinyin'])
                                        
                    # default custom styling type to phrase (which doesn't indent)                    
                    with tag('dp'):
                        with tag('ul'):
                            if 'synonyms' in h:
                                line('li', '相似詞：' + h['synonyms'])
                            if 'antonyms' in h:
                                line('li', '相反詞：' + h['antonyms'])

                            # process each individual definition for each heteronym
                            def_txt = h['definitions'][0]['defn']
                            # remove any ordered numberings
                            def_txt = re.sub(r'\d+[.]', '', def_txt)
                            # split by line
                            def_lst = def_txt.split('\n')
                            # get rid of empty elements
                            def_lst = [x for x in def_lst if x]

                            # whole definition should be in a li
                            with tag('li'):
                                doc.asis('釋義:<br>')
                                if len(def_lst) > 1:
                                    # order defs
                                    with tag('ol'):
                                        for d in def_lst:
                                            line('li', d)
                                else:
                                    text(def_lst[0])
                            
            
                                

        # always close each entry with hr
        doc.stag('hr')

Method 2
* indents data
* pastes in original def, no splitting and redoing into ordered list
* 20210802 08:15

In [ ]:
from yattag import Doc
from yattag import indent

doc, tag, text, line = Doc().ttl()

# 當:98775:98790
for j in range(len(data)):
    print(j)
    entry = data[j]
#     print(entry)
    # 'title' - searched character(s)
    # make sure not 異體字 which doesn't have a title
    if '{[' not in entry['title']:
        single_char = False
        
        if len(entry['title']) == 1:
            single_char = True
            if 'radical' in entry:
                radical = entry['radical']
            if 'non_radical_stroke_count' in entry:
                non_radical_stroke_count = entry['non_radical_stroke_count']
            if 'stroke_count' in entry:
                stroke_count = entry['stroke_count']
        
        # create entry
        with tag('idx:entry', name="default", scriptable="yes", spell="yes"):
            # create title
            with tag('h5'):
                # custom styling for title
                with tag('dt'):
                    with tag('idx:orth'):
                        text(entry['title'])
                    # add extra info for single characters
                    if single_char:
                        text('    (部首：{}, 部首外筆畫數：{}, 總筆畫數：{})'.format(radical, non_radical_stroke_count, stroke_count))
            # create definition for each heteronym (pronunciation -- typically more for single characters)
            for h in entry['heteronyms']:
                # custom styling for heteronym
                with tag('dh'):
                    # add pronunciation if exists (assume bpmf and pinyin always come together)
                    if 'bopomofo' in h and 'pinyin' in h:
                        text(h['bopomofo'])
                        doc.asis('<br>')
                        text(h['pinyin'])
                        doc.asis('<br>')
                                        
                    # default custom styling type to phrase (which doesn't indent)                    
                    with tag('dp'):

                        # process each individual definition for each heteronym
                        if 'synonyms' in h:
                            line('ds', '相似詞')
                            # doc.asis('<br>')
                            line('p', h['synonyms'])
                            
                        if 'antonyms' in h:
                            line('ds', '相反詞')
                            # doc.asis('<br>')
                            line('p', h['antonyms'])

                        # process each individual definition for each heteronym
                        def_txt = h['definitions'][0]['defn']
                        def_txt = def_txt.replace('\n', '<br>')
                        line('ds', '釋義')
                        
                        with tag('p'):
                            doc.asis(def_txt)

        # always close each entry with hr
        doc.stag('hr')

Method 2.5
- like above and still pastes in whole chunk
- but italicizes examples by breaking apart line by line then rejoining

In [ ]:
from yattag import Doc
from yattag import indent

doc, tag, text, line = Doc().ttl()

for j in range(len(data)):
    print(j)
    entry = data[j]

    # 'title' - searched character(s)
    if '{[' not in entry['title']:
        single_char = False
        
        if len(entry['title']) == 1:
            single_char = True
            if 'radical' in entry:
                radical = entry['radical']
            if 'non_radical_stroke_count' in entry:
                non_radical_stroke_count = entry['non_radical_stroke_count']
            if 'stroke_count' in entry:
                stroke_count = entry['stroke_count']
        
        # create entry
        with tag('idx:entry', name="default", scriptable="yes", spell="yes"):
            # create title
            with tag('h5'):
                # custom styling for title
                with tag('dt'):
                    with tag('idx:orth'):
                        text(entry['title'])
                    # add extra info for single characters
                    if single_char:
                        text('    (部首：{}, 部首外筆畫數：{}, 總筆畫數：{})'.format(radical, non_radical_stroke_count, stroke_count))
            # create definition for each heteronym (pronunciation -- typically more for single characters)
            for h in entry['heteronyms']:
                # custom styling for heteronym
                with tag('dh'):
                    # add pronunciation if exists (assume bpmf and pinyin always come together)
                    if 'bopomofo' in h and 'pinyin' in h:
                        text(h['bopomofo'])
                        doc.asis('<br>')
                        text(h['pinyin'])
                        doc.asis('<br>')
                                        
                    # default custom styling type to phrase (which doesn't indent)                    
                    with tag('dp'):
                        if 'synonyms' in h:
                            line('ds', '相似詞')
                            line('p', h['synonyms'])
                            
                        if 'antonyms' in h:
                            line('ds', '相反詞')
                            line('p', h['antonyms'])

                        # process each individual definition (they are all crammed in this first element of 'definitions')
                        def_txt = h['definitions'][0]['defn']
                        # split by line
                        def_lst = def_txt.split('\n')
                        # get rid of empty elements
                        def_lst = [x for x in def_lst if x]

                        line('ds', '釋義')
                        if len(def_lst) > 1:
                            for i in range(len(def_lst)):
                                d = def_lst[i]
                                # italicize the examples first
                                if '例:' in d:
                                    ind = d.find('例:')
                                    d = d[0:ind] + '<i>' + d[ind:] + '</i>'
                                
                                def_lst[i] = d
                            
                            def_txt = '<br>'.join(def_lst)
                            with tag('p'):
                                doc.asis(def_txt)

                        # double check to make sure list isn't empty (don't think
                        # it could be)
                        elif len(def_lst) == 1:
                            d = def_lst[0]
                            if '例:' in d:
                                ind = d.find('例:')
                                d = d[0:ind] + '<i>' + d[ind:] + '</i>'
                            with tag('p'):
                                doc.asis(d)

        # always close each entry with hr
        doc.stag('hr')

Method 3
* like above, but with ordered entries in `<ol>` and unordered in `<ul>` (trust that all ol comes before ul)
* italicizes examples

In [ ]:
from yattag import Doc
from yattag import indent

doc, tag, text, line = Doc().ttl()

for j in range(len(data)):
    print(j)
    entry = data[j]

    # 'title' - searched character(s)
    if '{[' not in entry['title']:
        single_char = False
        
        if len(entry['title']) == 1:
            single_char = True
            if 'radical' in entry:
                radical = entry['radical']
            if 'non_radical_stroke_count' in entry:
                non_radical_stroke_count = entry['non_radical_stroke_count']
            if 'stroke_count' in entry:
                stroke_count = entry['stroke_count']
        
        # create entry
        with tag('idx:entry', name="default", scriptable="yes", spell="yes"):
            # create title
            with tag('h5'):
                # custom styling for title
                with tag('dt'):
                    with tag('idx:orth'):
                        text(entry['title'])
                    # add extra info for single characters
                    if single_char:
                        text('    (部首：{}, 部首外筆畫數：{}, 總筆畫數：{})'.format(radical, non_radical_stroke_count, stroke_count))
            # create definition for each heteronym (pronunciation -- typically more for single characters)
            for h in entry['heteronyms']:
                # custom styling for heteronym
                with tag('dh'):
                    # add pronunciation if exists (assume bpmf and pinyin always come together)
                    if 'bopomofo' in h and 'pinyin' in h:
                        text(h['bopomofo'])
                        doc.asis('<br>')
                        text(h['pinyin'])
                        doc.asis('<br>')
                                        
                    # default custom styling type to phrase (which doesn't indent)                    
                    with tag('dp'):
                        if 'synonyms' in h:
                            line('ds', '相似詞')
                            line('p', h['synonyms'])
                            
                        if 'antonyms' in h:
                            line('ds', '相反詞')
                            line('p', h['antonyms'])

                        # process each individual definition (they are all crammed in this first element of 'definitions')
                        def_txt = h['definitions'][0]['defn']
                        # split by line
                        def_lst = def_txt.split('\n')
                        # get rid of empty elements
                        def_lst = [x for x in def_lst if x]

                        line('ds', '釋義')
                        if len(def_lst) > 1:
                            ordered = []
                            unordered = []
                            for d in def_lst:
                                # italicize the examples first
                                if '例:' in d:
                                    ind = d.find('例:')
                                    d = d[0:ind] + '<i>' + d[ind:] + '</i>'
                                
                                m = re.search(r'\d+[.]', d)
                                if m:
                                    ordered.append(d[len(m.group(0)):])
                                else:
                                    unordered.append(d)
                                    
                            with tag('ol'):
                                for o in ordered:
                                    with tag('li'):
                                        doc.asis(o)
                            with tag('ul'):
                                for u in unordered:
                                    with tag('li'):
                                        doc.asis(u)
                                

                        # double check to make sure list isn't empty (don't think
                        # it could be)
                        elif len(def_lst) == 1:
                            d = def_lst[0]
                            if '例:' in d:
                                ind = d.find('例:')
                                d = d[0:ind] + '<i>' + d[ind:] + '</i>'
                            with tag('p'):
                                doc.asis(d)

        # always close each entry with hr
        doc.stag('hr')
#     if j % 100 == 0:
#         f = open('generated.html', 'w', encoding='utf-8')
#         f.write(indent(doc.getvalue()))
#         f.close
#         print('saved')
    


Method 4
* like above but uses styling to differentiate between ordered and unordered in the same `<ol>`
* has headers, indents data
* uses `<ol>` and removes original numbering for alignment purposes
* italicizes example parts

In [ ]:
from yattag import Doc
from yattag import indent

doc, tag, text, line = Doc().ttl()

for j in range(len(data)):
    print(j)
    entry = data[j]

    # 'title' - searched character(s)
    if '{[' not in entry['title']:
        single_char = False
        
        if len(entry['title']) == 1:
            single_char = True
            if 'radical' in entry:
                radical = entry['radical']
            if 'non_radical_stroke_count' in entry:
                non_radical_stroke_count = entry['non_radical_stroke_count']
            if 'stroke_count' in entry:
                stroke_count = entry['stroke_count']
        
        # create entry
        with tag('idx:entry', name="default", scriptable="yes", spell="yes"):
            # create title
            with tag('h5'):
                # custom styling for title
                with tag('dt'):
                    with tag('idx:orth'):
                        text(entry['title'])
                    # add extra info for single characters
                    if single_char:
                        text('    (部首：{}, 部首外筆畫數：{}, 總筆畫數：{})'.format(radical, non_radical_stroke_count, stroke_count))
            # create definition for each heteronym (pronunciation -- typically more for single characters)
            for h in entry['heteronyms']:
                # custom styling for heteronym
                with tag('dh'):
                    # add pronunciation if exists (assume bpmf and pinyin always come together)
                    if 'bopomofo' in h and 'pinyin' in h:
                        text(h['bopomofo'])
                        doc.asis('<br>')
                        text(h['pinyin'])
                        doc.asis('<br>')
                                        
                    # default custom styling type to phrase (which doesn't indent)                    
                    with tag('dp'):
                        if 'synonyms' in h:
                            line('ds', '相似詞')
                            line('p', h['synonyms'])
                            
                        if 'antonyms' in h:
                            line('ds', '相反詞')
                            line('p', h['antonyms'])

                        # process each individual definition (they are all crammed in this first element of 'definitions')
                        def_txt = h['definitions'][0]['defn']
                        # split by line
                        def_lst = def_txt.split('\n')
                        # get rid of empty elements
                        def_lst = [x for x in def_lst if x]

                        line('ds', '釋義')
                        if len(def_lst) > 1:
                            with tag('ol'):
                                for d in def_lst:
                                    # italicize the examples first
                                    if '例:' in d:
                                        ind = d.find('例:')
                                        d = d[0:ind] + '<i>' + d[ind:] + '</i>'

                                    m = re.search(r'\d+[.]', d)
                                    if m:
                                        with tag('li'):
                                            d = d[len(m.group(0)):]
                                            doc.asis(d)
                                    else:
                                        with tag('li', klass='bulletstyle'):
                                            doc.asis(d)

                        # double check to make sure list isn't empty (don't think
                        # it could be)
                        elif len(def_lst) == 1:
                            d = def_lst[0]
                            if '例:' in d:
                                ind = d.find('例:')
                                d = d[0:ind] + '<i>' + d[ind:] + '</i>'
                            with tag('p'):
                                doc.asis(d)

        # always close each entry with hr
        doc.stag('hr')
#     if j % 100 == 0:
#         f = open('generated.html', 'w', encoding='utf-8')
#         f.write(indent(doc.getvalue()))
#         f.close
#         print('saved')
    


In [ ]:
f_template = open('content_template.html', 'r', encoding='utf-8')
s_template = f_template.read()
s_template = s_template.replace('@BODY', (doc.getvalue()))

f = open('content.html', 'w', encoding='utf-8')
f.write(s_template)
f.close
print('file saved')

In [ ]:
# can use this but no output, so I prefer using console directly
# !kindlepreviewer dict.opf -convert -output output/

v1 20210802 0815
- using method 2
- has indent in first line of definition but not others

v2 20210802 1055
- using method 2.5
- same as above but italicize examples and fix weird "only first line of `<p>` indented" problem in v1 by using margin instead of padding
- run ended 13:41

v3 20210802 1742
- using method 3
- same as above but tries to use separate `<ol>, <ul>` so multi-line defs are indented better
- syn and ant entries are too indented, change from 1em to 0.5em next run

v4 20210802 2021
- almost same as above, method 3
- change `<p>` left padding to 0.5em from 1em